In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Wikipedia GDP 페이지 요청
def extract_gdp(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 테이블 찾기
    table = soup.find('table', {'class': 'wikitable'})

    # 테이블 파싱
    rows = table.find_all('tr')
    data = []

    for row in rows[1:]:  # 헤더 제외
        cols = row.find_all(['td', 'th'])
        if len(cols) >= 4:
            country = cols[0].get_text(strip=True)  # 첫번째 컬럼에 나라 이름 저장
            imf_forecast = cols[1].get_text(strip=True).replace(",", "")    # 두번째 컬럼에 GDP 예측값 저장
            imf_year = cols[2].get_text(strip=True) # 세번째 컬럼에 예측 연도 저장
            
            # GDP 예측값이 숫자가 아니면 스킵
            try:
                imf_forecast = int(imf_forecast)
            except:
                continue 

            data.append({   # 데이터 프레임에 각 값 저장
                "Country": country,
                "IMF Forecast (Million USD)": imf_forecast,
                "IMF Year": imf_year
            })
    return data

# DataFrame 생성
data = extract_gdp("https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)")
df = pd.DataFrame(data)
df

,Country,IMF Forecast (Million USD),IMF Year
0,World,113795678,2025
1,United States,30507217,2025
2,China,19231705,[n 1]2025
3,Germany,4744804,2025
4,India,4187017,2025
...,...,...,...
185,Palau,333,2025
186,Kiribati,312,2025
187,Marshall Islands,297,2025
188,Nauru,169,2025


In [14]:
df['IMF Forecast (Million USD)'] /= 1000    # 원래 Million이었던 단위를 Billion으로 바꿔줌

In [15]:
df['IMF Forecast (Million USD)'] = df['IMF Forecast (Million USD)'].round(2)    # 소수점 2번째 자리까지만 출력

In [16]:
df.rename(columns={'IMF Forecast (Million USD)':'IMF Forecast (Billion USD)'}, inplace=True)    # 단위에 맞게 컬럼명을 Billion으로 변경

In [17]:
df

,Country,IMF Forecast (Billion USD),IMF Year
0,World,113795.68,2025
1,United States,30507.22,2025
2,China,19231.71,[n 1]2025
3,Germany,4744.80,2025
4,India,4187.02,2025
...,...,...,...
185,Palau,0.33,2025
186,Kiribati,0.31,2025
187,Marshall Islands,0.30,2025
188,Nauru,0.17,2025


In [18]:
GDP_above_100B = df[df['IMF Forecast (Billion USD)'] >= 100]    # GDP가 100B USD 이상인 국가
GDP_above_100B

,Country,IMF Forecast (Billion USD),IMF Year
0,World,113795.68,2025
1,United States,30507.22,2025
2,China,19231.71,[n 1]2025
3,Germany,4744.80,2025
4,India,4187.02,2025
...,...,...,...
68,Bulgaria,117.01,2025
69,Angola,113.34,2025
70,Venezuela,108.51,2025
71,Oman,104.35,2025


In [19]:
print(df.columns.tolist())

['Country', 'IMF Forecast (Billion USD)', 'IMF Year']


In [20]:
region_df = pd.read_csv('continents.csv')   # 국가와 지역이 매핑되어 있는 외부 csv 파일 활용

# 컬럼 이름 공백 제거 (혹시 모를 오타 방지용)
df.rename(columns=lambda x: x.strip(), inplace=True)
region_df.rename(columns=lambda x: x.strip(), inplace=True)

# 병합 수행
df = pd.merge(df, region_df[['Continent', 'name']], left_on='Country', right_on='name', how='left')
df.drop(columns=['name'], inplace=True)



In [21]:
# 결과 확인
df[['Country', 'IMF Forecast (Billion USD)', 'Continent']].head()

,Country,IMF Forecast (Billion USD),Continent
0,World,113795.68,NaN
1,United States,30507.22,North America
2,China,19231.71,Asia
3,Germany,4744.80,Europe
4,India,4187.02,Asia


In [22]:
# GDP 상위 5개 국가를 region별로 추출
df_top5 = df.sort_values(by='IMF Forecast (Billion USD)', ascending=False).groupby('Continent').head(5)

# region별 평균 GDP 계산
avg_gdp_by_region = df_top5.groupby('Continent')['IMF Forecast (Billion USD)'].mean()

avg_gdp_by_region

Continent
Africa            276.134
Asia             6294.376
Europe           3203.528
North America    6934.842
Oceania           412.270
South America     776.874
Name: IMF Forecast (Billion USD), dtype: float64